In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens/users.dat
/kaggle/input/movielens/ratings.dat
/kaggle/input/movielens/README
/kaggle/input/movielens/movies.dat
/kaggle/input/imdb-dataset/title.basics.tsv
/kaggle/input/imdb-dataset/title.principals.tsv
/kaggle/input/imdb-dataset/title.ratings.tsv
/kaggle/input/imdb-dataset/name.basics.tsv
/kaggle/input/imdb-dataset/title.crew.tsv


In [2]:
from scipy.sparse import csr_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from typing import List, Tuple
import random
import time

2024-07-07 16:28:15.989598: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 16:28:15.989731: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 16:28:16.134407: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# # MovieLens datasets

mlens_movies_dtypes = {
    "movieId": "int64",
    "title": "string",
    "genres": "string"
}

mlens_ratings_dtypes = {
    "userId": "int64",
    "movieId": "int64",
    "rating": "int64",
    "timestamp": "int64"
}

mlens_users_dtypes = {
    "userId": "int64",
    "sex": "string",
    "age": "int64",
    "occupation": "int64",
    "zipcode": "int64"
}

mlens_movies_df = pd.read_csv("/kaggle/input/movielens/movies.dat", sep='::', header=None, engine='python', dtype=mlens_movies_dtypes, na_values="\\N", encoding='latin-1')
mlens_ratings_df = pd.read_csv("/kaggle/input/movielens/ratings.dat", sep='::', header=None, engine='python', dtype=mlens_ratings_dtypes, na_values="\\N")
mlens_users_df = pd.read_csv("/kaggle/input/movielens/users.dat", sep='::', header=None, engine='python', dtype=mlens_users_dtypes, na_values="\\N")
mlens_movies_df.rename(columns={0: 'movieId', 1: 'title', 2:'genres'}, inplace=True)
mlens_ratings_df.rename(columns={0: 'userId', 1:'movieId', 2:'rating', 3:'timestamp'}, inplace=True)
mlens_users_df.rename(columns={0: 'userId', 1: 'sex', 2: 'age', 3: 'occupation', 4: 'zipcode'}, inplace=True)

In [4]:
# import datasets
# IMDb datasets
imdb_titles_dtypes = {
    "tconst": "string",
    "titleType": "string",
    "primaryTitle": "string",
    "originalTitle": "string",
    "isAdult": "string",
    "startYear": "Int64",  # using Int64 to handle NaNs
    "endYear": "Int64",    # using Int64 to handle NaNs
    "runtimeMinutes": "string",  # using Int64 to handle NaNs
    "genres": "string"
}

imdb_names_dtypes = {
    "nconst": "string",
    "primaryName": "string",
    "birthYear": "Int64",  # using Int64 to handle NaNs
    "deathYear": "Int64",  # using Int64 to handle NaNs
    "primaryProfession": "string",
    "knownForTitles": "string"
}

imdb_ratings_dtypes = {
    "tconst": "string",
    "averageRating": "float64",
    "numVotes": "int64"
}

imdb_principals_dtypes = {
    "tconst": "string",
    "ordering": "int64",
    "nconst": "string",
    "category": "string",
    "job": "string",
    "characters": "string"
}

imdb_crew_dtypes = {
    "tconst": "string",
    "directors": "string",
    "writers": "string"
}

# IMDb datasets
imdb_titles_df = pd.read_csv("/kaggle/input/imdb-dataset/title.basics.tsv", sep="\t", header=0, dtype=imdb_titles_dtypes, na_values="\\N")
# imdb_names_df = pd.read_csv("/kaggle/input/imdb-dataset/name.basics.tsv", sep="\t", header=0, dtype=imdb_names_dtypes, na_values="\\N")
imdb_ratings_df = pd.read_csv("/kaggle/input/imdb-dataset/title.ratings.tsv", sep="\t", header=0, dtype=imdb_ratings_dtypes, na_values="\\N")
# imdb_principals_df = pd.read_csv("/kaggle/input/imdb-dataset/title.principals.tsv", sep="\t", header=0, dtype=imdb_principals_dtypes, na_values="\\N")
# imdb_crew_df = pd.read_csv("/kaggle/input/imdb-dataset/title.crew.tsv", sep="\t", header=0, dtype=imdb_crew_dtypes, na_values="\\N")

In [5]:
imdb_titles_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,<NA>,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,<NA>,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,<NA>,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,<NA>,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,<NA>,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
10900922,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2009,<NA>,<NA>,"Action,Drama,Family"
10900923,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,<NA>,<NA>,"Action,Drama,Family"
10900924,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,<NA>,<NA>,"Action,Drama,Family"
10900925,tt9916856,short,The Wind,The Wind,0,2015,<NA>,27,Short


In [6]:
# removing rows with error on the columns (shift of the values to the left because originalTitle was put with primaryTitle so everything is shifted)
imdb_titles_df = imdb_titles_df[imdb_titles_df['genres'].notna()]
imdb_titles_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,<NA>,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,<NA>,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,<NA>,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,<NA>,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,<NA>,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
10900922,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2009,<NA>,<NA>,"Action,Drama,Family"
10900923,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,<NA>,<NA>,"Action,Drama,Family"
10900924,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,<NA>,<NA>,"Action,Drama,Family"
10900925,tt9916856,short,The Wind,The Wind,0,2015,<NA>,27,Short


In [7]:
# checking all titleTypes
imdb_titles_df['titleType'].unique()

<StringArray>
[       'short',        'movie',      'tvShort',      'tvMovie',
     'tvSeries',    'tvEpisode', 'tvMiniSeries',    'tvSpecial',
        'video',    'videoGame']
Length: 10, dtype: string

In [8]:
# keeping only movies / tvMovies
imdb_titles_df_movies = imdb_titles_df[imdb_titles_df['titleType'].str.contains('ovie')] 
imdb_titles_df_movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,<NA>,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,<NA>,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,<NA>,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,<NA>,90,Drama
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,<NA>,<NA>,Drama
...,...,...,...,...,...,...,...,...,...
10900845,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,<NA>,100,Documentary
10900850,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,0,2015,<NA>,66,Drama
10900857,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013,<NA>,<NA>,Comedy
10900867,tt9916730,movie,6 Gunn,6 Gunn,0,2017,<NA>,116,Drama


In [9]:
# removing na values and casting to right type
imdb_titles_df_clean = imdb_titles_df_movies.drop(columns=['endYear', 'originalTitle'])
imdb_titles_df_clean = imdb_titles_df_clean[imdb_titles_df_clean['startYear'].notna()]
imdb_titles_df_clean = imdb_titles_df_clean[imdb_titles_df_clean['runtimeMinutes'].notna()]
imdb_titles_df_clean['isAdult'] = imdb_titles_df_clean['isAdult'].astype(int)
imdb_titles_df_clean = imdb_titles_df_clean[imdb_titles_df_clean['isAdult'] <= 1]
imdb_titles_df_clean['isAdult'] = imdb_titles_df_clean['isAdult'] == 1
imdb_titles_df_clean['runtineMinutes'] = imdb_titles_df_clean['runtimeMinutes'].astype(int)
imdb_titles_df_clean['startYear'] = imdb_titles_df_clean['startYear'].astype(int)
imdb_titles_df_clean

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,runtineMinutes
8,tt0000009,movie,Miss Jerry,False,1894,45,Romance,45
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,False,1897,100,"Documentary,News,Sport",100
570,tt0000574,movie,The Story of the Kelly Gang,False,1906,70,"Action,Adventure,Biography",70
587,tt0000591,movie,The Prodigal Son,False,1907,90,Drama,90
672,tt0000679,movie,The Fairylogue and Radio-Plays,False,1908,120,"Adventure,Fantasy",120
...,...,...,...,...,...,...,...,...
10900818,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,False,2015,57,Documentary,57
10900845,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,False,2007,100,Documentary,100
10900850,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,False,2015,66,Drama,66
10900867,tt9916730,movie,6 Gunn,False,2017,116,Drama,116


In [10]:
# two different movies with same title -> let's keep them
imdb_titles_df_clean[imdb_titles_df_clean['primaryTitle'] == 'Bohemios']

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,runtineMinutes
30557,tt0031112,movie,Bohemios,False,1939,80,Musical,80
215673,tt0225200,movie,Bohemios,False,1935,62,Drama,62


In [11]:
imdb_ratings_df

,tconst,averageRating,numVotes
0,tt0000001,5.7,2061
1,tt0000002,5.6,278
2,tt0000003,6.5,2028
3,tt0000004,5.4,179
4,tt0000005,6.2,2796
...,...,...,...
1452482,tt9916730,7.0,12
1452483,tt9916766,7.1,23
1452484,tt9916778,7.2,37
1452485,tt9916840,7.2,10


In [12]:
len(imdb_ratings_df), len(imdb_ratings_df['numVotes'].notnull()),len(imdb_ratings_df['numVotes'] >= 50)

(1452487, 1452487, 1452487)

In [13]:
# merging the movies and their ratings info
imdb_df = pd.merge(imdb_titles_df_clean, imdb_ratings_df, how='inner', left_on='tconst', right_on='tconst')
imdb_df

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,runtineMinutes,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,False,1894,45,Romance,45,5.4,211
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,False,1897,100,"Documentary,News,Sport",100,5.2,517
2,tt0000574,movie,The Story of the Kelly Gang,False,1906,70,"Action,Adventure,Biography",70,6.0,912
3,tt0000591,movie,The Prodigal Son,False,1907,90,Drama,90,5.6,26
4,tt0000679,movie,The Fairylogue and Radio-Plays,False,1908,120,"Adventure,Fantasy",120,5.3,74
...,...,...,...,...,...,...,...,...,...,...
319807,tt9916192,tvMovie,Danielle Darrieux: Il est poli d'être gai!,False,2019,53,Biography,53,7.3,18
319808,tt9916270,movie,Il talento del calabrone,False,2020,84,Thriller,84,5.8,1489
319809,tt9916362,movie,Coven,False,2020,92,"Drama,History",92,6.4,5844
319810,tt9916538,movie,Kuambil Lagi Hatiku,False,2019,123,Drama,123,8.3,9


In [14]:
# ensuring the same format for the title in order to merge the datasets
imdb_df['primaryTitle'] = imdb_df['primaryTitle'].str.lower()
imdb_df

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,runtineMinutes,averageRating,numVotes
0,tt0000009,movie,miss jerry,False,1894,45,Romance,45,5.4,211
1,tt0000147,movie,the corbett-fitzsimmons fight,False,1897,100,"Documentary,News,Sport",100,5.2,517
2,tt0000574,movie,the story of the kelly gang,False,1906,70,"Action,Adventure,Biography",70,6.0,912
3,tt0000591,movie,the prodigal son,False,1907,90,Drama,90,5.6,26
4,tt0000679,movie,the fairylogue and radio-plays,False,1908,120,"Adventure,Fantasy",120,5.3,74
...,...,...,...,...,...,...,...,...,...,...
319807,tt9916192,tvMovie,danielle darrieux: il est poli d'être gai!,False,2019,53,Biography,53,7.3,18
319808,tt9916270,movie,il talento del calabrone,False,2020,84,Thriller,84,5.8,1489
319809,tt9916362,movie,coven,False,2020,92,"Drama,History",92,6.4,5844
319810,tt9916538,movie,kuambil lagi hatiku,False,2019,123,Drama,123,8.3,9


## MOVIELENS

In [15]:
mlens_movies_df

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [16]:
# ensuring the same format as the imdb dataset and extracting the startYear info
mlens_movies_df['startYear'] = mlens_movies_df['title'].str[-5:-1].astype(int)
mlens_movies_df['title'] = mlens_movies_df['title'].str[:-7]
mlens_movies_df['title'] = mlens_movies_df['title'].str.lower()
mlens_movies_df

,movieId,title,genres,startYear
0,1,toy story,Animation|Children's|Comedy,1995
1,2,jumanji,Adventure|Children's|Fantasy,1995
2,3,grumpier old men,Comedy|Romance,1995
3,4,waiting to exhale,Comedy|Drama,1995
4,5,father of the bride part ii,Comedy,1995
...,...,...,...,...
3878,3948,meet the parents,Comedy,2000
3879,3949,requiem for a dream,Drama,2000
3880,3950,tigerland,Drama,2000
3881,3951,two family house,Drama,2000


In [17]:
mlens_ratings_df

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [18]:
# dropping useless column timestamp
mlens_ratings_df = mlens_ratings_df.drop(columns='timestamp')
mlens_ratings_df

,userId,movieId,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [19]:
ratings_df = mlens_ratings_df
ratings_df

,userId,movieId,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


## MERGING

In [20]:
# merging based on title
movies_df = pd.merge(mlens_movies_df, imdb_df, how='inner', left_on=['title'], right_on=['primaryTitle'])
movies_df

,movieId,title,genres_x,startYear_x,tconst,titleType,primaryTitle,isAdult,startYear_y,runtimeMinutes,genres_y,runtineMinutes,averageRating,numVotes
0,1,toy story,Animation|Children's|Comedy,1995,tt0114709,movie,toy story,False,1995,81,"Adventure,Animation,Comedy",81,8.3,1075934
1,2,jumanji,Adventure|Children's|Fantasy,1995,tt0113497,movie,jumanji,False,1995,104,"Adventure,Comedy,Family",104,7.1,379991
2,3,grumpier old men,Comedy|Romance,1995,tt0113228,movie,grumpier old men,False,1995,101,"Comedy,Romance",101,6.6,29870
3,4,waiting to exhale,Comedy|Drama,1995,tt0114885,movie,waiting to exhale,False,1995,124,"Comedy,Drama,Romance",124,6.0,12307
4,5,father of the bride part ii,Comedy,1995,tt0113041,movie,father of the bride part ii,False,1995,106,"Comedy,Family,Romance",106,6.1,41935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5278,3948,meet the parents,Comedy,2000,tt0212338,movie,meet the parents,False,2000,108,"Comedy,Romance",108,7.0,357316
5279,3949,requiem for a dream,Drama,2000,tt0180093,movie,requiem for a dream,False,2000,102,Drama,102,8.3,904118
5280,3950,tigerland,Drama,2000,tt0170691,movie,tigerland,False,2000,101,"Drama,War",101,6.9,43498
5281,3950,tigerland,Drama,2000,tt7913478,movie,tigerland,False,2019,91,Documentary,91,6.9,72


In [21]:
# cleaning the merged dataset
movies_df = movies_df.drop(columns=['runtimeMinutes', 'primaryTitle'])
movies_df['genres_x'] = movies_df['genres_x'].str.split("|")
movies_df['genres_y'] = movies_df['genres_y'].str.split(",")
movies_df['genres'] = movies_df['genres_x'] + movies_df['genres_y']
movies_df = movies_df.drop(columns=['genres_x', 'genres_y'])
movies_df

,movieId,title,startYear_x,tconst,titleType,isAdult,startYear_y,runtineMinutes,averageRating,numVotes,genres
0,1,toy story,1995,tt0114709,movie,False,1995,81,8.3,1075934,"[Animation, Children's, Comedy, Adventure, Ani..."
1,2,jumanji,1995,tt0113497,movie,False,1995,104,7.1,379991,"[Adventure, Children's, Fantasy, Adventure, Co..."
2,3,grumpier old men,1995,tt0113228,movie,False,1995,101,6.6,29870,"[Comedy, Romance, Comedy, Romance]"
3,4,waiting to exhale,1995,tt0114885,movie,False,1995,124,6.0,12307,"[Comedy, Drama, Comedy, Drama, Romance]"
4,5,father of the bride part ii,1995,tt0113041,movie,False,1995,106,6.1,41935,"[Comedy, Comedy, Family, Romance]"
...,...,...,...,...,...,...,...,...,...,...,...
5278,3948,meet the parents,2000,tt0212338,movie,False,2000,108,7.0,357316,"[Comedy, Comedy, Romance]"
5279,3949,requiem for a dream,2000,tt0180093,movie,False,2000,102,8.3,904118,"[Drama, Drama]"
5280,3950,tigerland,2000,tt0170691,movie,False,2000,101,6.9,43498,"[Drama, Drama, War]"
5281,3950,tigerland,2000,tt7913478,movie,False,2019,91,6.9,72,"[Drama, Documentary]"


In [22]:
movies_df = movies_df[movies_df['startYear_x'] == movies_df['startYear_y']]
movies_df = movies_df.drop(columns=['startYear_y', 'tconst']).rename(columns={'startYear_x': 'startYear'})
movies_df

,movieId,title,startYear,titleType,isAdult,runtineMinutes,averageRating,numVotes,genres
0,1,toy story,1995,movie,False,81,8.3,1075934,"[Animation, Children's, Comedy, Adventure, Ani..."
1,2,jumanji,1995,movie,False,104,7.1,379991,"[Adventure, Children's, Fantasy, Adventure, Co..."
2,3,grumpier old men,1995,movie,False,101,6.6,29870,"[Comedy, Romance, Comedy, Romance]"
3,4,waiting to exhale,1995,movie,False,124,6.0,12307,"[Comedy, Drama, Comedy, Drama, Romance]"
4,5,father of the bride part ii,1995,movie,False,106,6.1,41935,"[Comedy, Comedy, Family, Romance]"
...,...,...,...,...,...,...,...,...,...
5275,3947,get carter,1971,movie,False,112,7.3,37001,"[Thriller, Action, Crime, Thriller]"
5278,3948,meet the parents,2000,movie,False,108,7.0,357316,"[Comedy, Comedy, Romance]"
5279,3949,requiem for a dream,2000,movie,False,102,8.3,904118,"[Drama, Drama]"
5280,3950,tigerland,2000,movie,False,101,6.9,43498,"[Drama, Drama, War]"


In [23]:
movies_df_save = movies_df

In [24]:
movies_df_save['genres'] = movies_df_save['genres'].apply(lambda x: list(set(x)))
movies_df_exploded = movies_df.explode('genres')
df_onehot = pd.get_dummies(movies_df_exploded['genres'])
df_onehot = df_onehot.groupby(df_onehot.index).sum()
movies_df_final = movies_df_save.drop(columns='genres').join(df_onehot).groupby(['movieId', 'title', 'startYear', 'titleType', 'isAdult', 'runtineMinutes', 'averageRating', 'numVotes'], as_index=False).sum()
movies_df_final

,movieId,title,startYear,titleType,isAdult,runtineMinutes,averageRating,numVotes,Action,Adventure,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1,toy story,1995,movie,False,81,8.3,1075934,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,jumanji,1995,movie,False,104,7.1,379991,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,grumpier old men,1995,movie,False,101,6.6,29870,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,waiting to exhale,1995,movie,False,124,6.0,12307,0,0,...,0,0,0,0,1,0,0,0,0,0
4,5,father of the bride part ii,1995,movie,False,106,6.1,41935,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2426,3947,get carter,1971,movie,False,112,7.3,37001,1,0,...,0,0,0,0,0,0,0,1,0,0
2427,3948,meet the parents,2000,movie,False,108,7.0,357316,0,0,...,0,0,0,0,1,0,0,0,0,0
2428,3949,requiem for a dream,2000,movie,False,102,8.3,904118,0,0,...,0,0,0,0,0,0,0,0,0,0
2429,3950,tigerland,2000,movie,False,101,6.9,43498,0,0,...,0,0,0,0,0,0,0,0,1,0


In [25]:
movies_df_final.columns

Index(['movieId', 'title', 'startYear', 'titleType', 'isAdult',
       'runtineMinutes', 'averageRating', 'numVotes', 'Action', 'Adventure',
       'Animation', 'Biography', 'Children's', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport',
       'Thriller', 'War', 'Western'],
      dtype='object')

In [26]:
# keeping only the ratings of the movies present in movies_df_final
filtered_ratings_df = ratings_df[ratings_df['movieId'].isin(movies_df_final['movieId'])]
filtered_ratings_df

,userId,movieId,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
6,1,1287,5
...,...,...,...
1000203,6040,1090,3
1000204,6040,1091,1
1000206,6040,562,5
1000207,6040,1096,4


In [27]:
ratings_matrix_df = filtered_ratings_df.pivot(index='userId', columns='movieId', values='rating').fillna(0)
ratings_matrix_df

movieId,1,2,3,4,5,6,7,8,9,10,...,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
movies_df = movies_df_final[movies_df_final['movieId'].isin(ratings_matrix_df.columns)]
movies_df.drop_duplicates(subset=['movieId'], inplace=True)

/tmp/ipykernel_33/1995180964.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df.drop_duplicates(subset=['movieId'], inplace=True)


In [29]:
mlens_users_df['male'] = (mlens_users_df['sex'] == 'M')
mlens_users_df.drop(columns='sex', inplace=True)

In [30]:
one_hot_encoded = pd.get_dummies(mlens_users_df['occupation'], prefix='occupation')
users_df = pd.concat([mlens_users_df, one_hot_encoded], axis=1)
users_df.drop('occupation', axis=1, inplace=True)
users_df

,userId,age,zipcode,male,occupation_0,occupation_1,occupation_2,occupation_3,occupation_4,occupation_5,...,occupation_11,occupation_12,occupation_13,occupation_14,occupation_15,occupation_16,occupation_17,occupation_18,occupation_19,occupation_20
0,1,1,48067,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,56,70072,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,3,25,55117,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,4,45,02460,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,5,25,55455,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,6036,25,32603,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
6036,6037,45,76006,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6037,6038,56,14706,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6038,6039,45,01060,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [31]:
users_df

,userId,age,zipcode,male,occupation_0,occupation_1,occupation_2,occupation_3,occupation_4,occupation_5,...,occupation_11,occupation_12,occupation_13,occupation_14,occupation_15,occupation_16,occupation_17,occupation_18,occupation_19,occupation_20
0,1,1,48067,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,56,70072,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,3,25,55117,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,4,45,02460,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,5,25,55455,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,6036,25,32603,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
6036,6037,45,76006,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6037,6038,56,14706,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6038,6039,45,01060,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Recommander Algorithms
### Bayesian

In [32]:
class TripletLossLayer(tf.keras.layers.Layer):
    """
    Layer object to minimise the triplet loss.
    Here we implement the Bayesian Personal Ranking triplet loss.
    """

    def __init__(self, **kwargs):
        super(TripletLossLayer, self).__init__(**kwargs)

    def bpr_triplet_loss(self, inputs):
        """
        Bayesian Personal Ranking triplet loss.
        We actually use log-loss for numerical purposes.
        """
        anchor, positive, negative = inputs
        p_dist = tf.math.reduce_sum(anchor * positive, axis=-1, keepdims=True)
        n_dist = tf.math.reduce_sum(anchor * negative, axis=-1, keepdims=True)
        return tf.math.log(1.0 - tf.math.sigmoid(p_dist - n_dist))

    def call(self, inputs):
        loss = self.bpr_triplet_loss(inputs)
        self.add_loss(loss)
        return loss

In [33]:
class ScoreLayer(tf.keras.layers.Layer):
    """
    Layer object to predict positive matches.
    """

    def __init__(self, **kwargs):
        super(ScoreLayer, self).__init__(**kwargs)

    def rec_similarity(self, inputs):
        """
        rec_similarity function
        """
        anchor, item = inputs
        score = tf.keras.layers.Dot(axes=1)([anchor, item])
        return score

    def call(self, inputs):
        pred = self.rec_similarity(inputs)
        return pred

In [34]:
def get_pos(df, anchor):
    POS_THR = 3.5
    ps = df.loc[anchor] >= POS_THR
    return ps[ps].index.values

def get_neg(df, anchor):
    NEG_THR = 2.0
    ps = df.loc[anchor] <= NEG_THR
    return ps[ps].index.values

In [35]:
user_con_features = ['age']
user_cat_features = ['male', 'occupation_0', 'occupation_1',
       'occupation_2', 'occupation_3', 'occupation_4', 'occupation_5',
       'occupation_6', 'occupation_7', 'occupation_8', 'occupation_9',
       'occupation_10', 'occupation_11', 'occupation_12', 'occupation_13',
       'occupation_14', 'occupation_15', 'occupation_16', 'occupation_17',
       'occupation_18', 'occupation_19', 'occupation_20']

item_con_features = ['startYear', 'runtineMinutes', 'averageRating', 'numVotes']
item_cat_features = ['isAdult', 'Action', 'Adventure',
       'Animation', 'Biography', 'Children\'s', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport',
       'Thriller', 'War', 'Western']

In [36]:
def process_features(df: pd.DataFrame, con_features: List[str], cat_features: List[str]) -> np.ndarray:
    X_cat = []
    X_con = []
    
    # continuous features are added directly
    for column in con_features:
        X = np.array(df[column])
        X_con.append(X)
    
    # categorical features are added after encoding
    for column in cat_features:
        X = np.asarray(df[column].tolist())
        X_line = pd.factorize(X)[0]
        X_cat.append(np.asarray(X_line))
        
    # transform lists in arrays
    try:
        X_con = np.column_stack(X_con)
    except:
        pass

    try:
        X_cat = np.column_stack(X_cat)
    except:
        pass

    # concatenate arrays
    if (len(cat_features) > 0) and (len(con_features) > 0):
        X = np.concatenate((X_con, X_cat), axis=1)
    elif len(cat_features) == 0:
        X = X_con
    elif len(con_features) == 0:
        X = X_cat
    # return the encoded features
    return X

In [37]:
X_usr = process_features(users_df, user_con_features, user_cat_features)
X_item = process_features(movies_df, item_con_features, item_cat_features)

## Model

In [92]:
def build_model(
    n_users: int,
    n_user_features: int,
    n_items: int,
    n_item_features: int,
    emb_dim: int = 32,
) -> Tuple[tf.keras.Model, tf.keras.Model]:
    """
    Define the Model for training.

    Parameters
    ----------

    n_users : int
                number of users
    n_items : int
                number of items
    emb_dim : int
                dimension of the embedding space
                default = 32

    Return
    ------
    model_train : tf.keras.Model
                Model instance to be trained
    model_predict : tf.keras.Model
                Model instance to be used for predictions
    """
    ### Input Layers
    user_input = tf.keras.Input((n_user_features,), name="user_input")
    positive_item_input = tf.keras.Input((n_item_features,), name="pos_item_input")
    negative_item_input = tf.keras.Input((n_item_features,), name="neg_item_input")

    inputs = [user_input, positive_item_input, negative_item_input]

    ### Embedding Layers
    # User embedding
    user_embedding = tf.keras.layers.Embedding(
        n_users, emb_dim, input_length=n_user_features, name="user_embedding"
    )
    # Positive and negative items will share the same embedding
    item_embedding = tf.keras.layers.Embedding(
        n_items, emb_dim, input_length=n_item_features, name="item_embedding"
    )
    # Layers to convert embedding vectors in the same dimensional vectors
    vec_conv64 = tf.keras.layers.Dense(64, name="dense_vec64", activation="relu")
    vec_conv32 = tf.keras.layers.Dense(32, name="dense_vec32", activation="relu")
    vec_conv = tf.keras.layers.Dense(emb_dim, name="dense_vec", activation="softmax")

    # Anchor (the user) embedding
    user_vec = tf.keras.layers.Flatten()(user_embedding(user_input))
    user_vec = tf.keras.layers.Dense(emb_dim, name="dense_user", activation="softmax")(
        user_vec
    )

    # Positive item embedding
    pos_item_vec = tf.keras.layers.Flatten()(item_embedding(positive_item_input))
    pos_item_vec = vec_conv64(pos_item_vec)
    pos_item_vec = vec_conv32(pos_item_vec)
    pos_item_vec = vec_conv(pos_item_vec)

    # Negative item embedding
    neg_item_vec = tf.keras.layers.Flatten()(item_embedding(negative_item_input))
    neg_item_vec = vec_conv64(neg_item_vec)
    neg_item_vec = vec_conv32(neg_item_vec)
    neg_item_vec = vec_conv(neg_item_vec)

    ### Score Layers
    p_rec_score = ScoreLayer(name="pos_recommendation_score")([user_vec, pos_item_vec])
    n_rec_score = ScoreLayer(name="neg_recommendation_score")([user_vec, neg_item_vec])

    ### Triplet Loss Layer
    loss = TripletLossLayer(name="triple_loss")([user_vec, pos_item_vec, neg_item_vec])

    # Connect the inputs with the outputs
    network_train = tf.keras.Model(inputs=inputs, outputs=loss, name="training_model")

    network_predict = tf.keras.Model(
        inputs=inputs[:-1], outputs=p_rec_score, name="inference_model"
    )

    # return the model
    return network_train, network_predict

In [39]:
n_users, n_items = ratings_matrix_df.shape
network_train, network_predict = build_model(
    n_users=n_users,
    n_user_features=X_usr.shape[1],
    n_items=n_items,
    n_item_features=X_item.shape[1],
    emb_dim=32,
)
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)
network_train.compile(loss=None, optimizer=optimiser)
network_train.summary()
tf.keras.utils.plot_model(
    network_train,
    show_shapes=True,
    show_layer_names=True,
    to_file="/kaggle/working/bpr_model.png",
)
n_iteration = 0

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "training_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ pos_item_input      │ (None, 28)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ neg_item_input      │ (None, 28)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_embedding      │ (None, 28, 32)    │     74,848 │ pos_item_input[0… │
│ (Embedding)         │                   │            │ neg_item_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_input          │ (None, 23)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 896)       │          0 │ item_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 896)       │          0 │ item_embedding[1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 23, 32)    │    193,280 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_vec64 (Dense) │ (None, 64)        │     57,408 │ flatten_1[0][0],  │
│                     │                   │            │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 736)       │          0 │ user_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_vec32 (Dense) │ (None, 32)        │      2,080 │ dense_vec64[0][0… │
│                     │                   │            │ dense_vec64[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_user (Dense)  │ (None, 32)        │     23,584 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_vec (Dense)   │ (None, 32)        │      1,056 │ dense_vec32[0][0… │
│                     │                   │            │ dense_vec32[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ triple_loss         │ (None, 1)         │          0 │ dense_user[0][0], │
│ (TripletLossLayer)  │                   │            │ dense_vec[0][0],  │
│                     │                   │            │ dense_vec[1][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 352,256 (1.34 MB)

 Trainable params: 352,256 (1.34 MB)

 Non-trainable params: 0 (0.00 B)

In [40]:
X_usr[0], X_item[0], X_item[8]

(array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0]),
 array([1.995000e+03, 8.100000e+01, 8.300000e+00, 1.075934e+06,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00]),
 array([1.9950e+03, 1.1100e+02, 5.8000e+00, 3.7256e+04, 0.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00]))

In [41]:
def cosine_dist(x, y, eps=1e-6):
    return 1 - np.dot(x, y.T) / (np.linalg.norm(x) * np.linalg.norm(y) + eps)

In [44]:
df_reset = movies_df.reset_index(drop=False)
movie_id_to_index_map = df_reset['movieId'].to_dict()
movie_id_to_index_map = {v: k for k, v in movie_id_to_index_map.items()}
    

def build_batch(batch_size, X_usr, X_item, df, return_cache=False):
    """
    Returns the list of three arrays to feed the model.

    Parameters
    ----------
    batch_size : int
                  size of the batch.

    X_usr : numpy array of shape (n_users, n_user_features)
                array of user metadata.

    X_item : numpy array of shape (n_items, n_item_features)
                array of item metadata.

    df : Pandas DataFrame
            dataframe containing user-item ratings.

    return_cache : bool
                    parameter to trigger whether we want the list of ids corresponding to triplets.
                    default: False

    Returns
    -------
    triplets : list of numpy arrays
                list containing 3 tensors A,P,N corresponding to:
                    - Anchor A : (batch_size, n_user_features)
                    - Positive P : (batch_size, n_item_features)
                    - Negative N : (batch_size, n_item_features)
    """
    # constant values
    n_user_features = X_usr.shape[1]
    n_item_features = X_item.shape[1]

    # define user_list
    user_list = list(df.index.values)

    # initialise result
    triplets = [
        np.zeros((batch_size, n_user_features)),  # anchor
        np.zeros((batch_size, n_item_features)),  # pos
        np.zeros((batch_size, n_item_features)),  # neg
    ]
    user_ids = []
    p_ids = []
    n_ids = []

    for i in range(batch_size):
        # pick one random user for anchor
        anchor_id = random.choice(user_list)
        user_ids.append(anchor_id)

        # all possible positive/negative samples for selected anchor
        p_item_ids = get_pos(df, anchor_id)
        n_item_ids = get_neg(df, anchor_id)

        # pick one of the positve ids
        try:
            positive_id = random.choice(p_item_ids)
        except IndexError:
            positive_id = 0

        p_ids.append(positive_id)

        # pick one of the negative ids
        try:
            negative_id = random.choice(n_item_ids)
        except IndexError:
            negative_id = 0

        n_ids.append(negative_id)

        # define triplet
        triplets[0][i, :] = X_usr[anchor_id][:]
        if positive_id == 0:
            triplets[1][i, :] = np.zeros((n_item_features,))
        else:
            triplets[1][i, :] = X_item[movie_id_to_index_map.get(positive_id)][:]

        if negative_id == 0:
            triplets[2][i, :] = np.zeros((n_item_features,))
        else:
            triplets[2][i, :] = X_item[movie_id_to_index_map.get(negative_id)][:]

    if return_cache:
        cache = {"users": user_ids, "positive": p_ids, "negative": n_ids}
        return triplets, cache

    return triplets

In [52]:
build_batch(batch_size=1, X_usr=X_usr, X_item=X_item, df=ratings_matrix_df, return_cache=True)

([array([[25.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[2.00000e+03, 1.09000e+02, 7.00000e+00, 2.54168e+05, 0.00000e+00,
          1.00000e+00, 1.00000e+00, 1.00000e+00, 0.00000e+00, 1.00000e+00,
          1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
          0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00,
          0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00,
          0.00000e+00, 0.00000e+00, 0.00000e+00]]),
  array([[1.9930e+03, 1.1400e+02, 6.3000e+00, 2.3332e+04, 0.0000e+00,
          0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
          1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00]])],
 {'users': [2697], 'positive': [330

In [228]:
movies_df

,movieId,title,startYear,titleType,isAdult,runtineMinutes,averageRating,numVotes,Action,Adventure,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1,toy story,1995,movie,False,81,8.3,1075934,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,jumanji,1995,movie,False,104,7.1,379991,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,grumpier old men,1995,movie,False,101,6.6,29870,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,waiting to exhale,1995,movie,False,124,6.0,12307,0,0,...,0,0,0,0,1,0,0,0,0,0
4,5,father of the bride part ii,1995,movie,False,106,6.1,41935,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2426,3947,get carter,1971,movie,False,112,7.3,37001,1,0,...,0,0,0,0,0,0,0,1,0,0
2427,3948,meet the parents,2000,movie,False,108,7.0,357316,0,0,...,0,0,0,0,1,0,0,0,0,0
2428,3949,requiem for a dream,2000,movie,False,102,8.3,904118,0,0,...,0,0,0,0,0,0,0,0,0,0
2429,3950,tigerland,2000,movie,False,101,6.9,43498,0,0,...,0,0,0,0,0,0,0,0,1,0


In [57]:
def get_triplets_hard(batch_size, X_usr, X_item, df, return_cache=False):
    """
    Returns the list of three arrays to feed the model.

    Parameters
    ----------
        batch_size : int
                        size of the batch.
        X_usr : numpy array of shape (n_users, n_user_features)
                        array of user metadata.
        X_item : numpy array of shape (n_items, n_item_features)
                        array of item metadata.
        df : Pandas DataFrame
                dataframe containing user-item ratings.
        return_cache : bool
                        parameter to trigger whether we want the list of ids corresponding to
                        triplets.
                        default: False

    Returns
    -------
        triplets : list of numpy arrays
                    list containing 3 tensors A,P,N corresponding to:
                        - Anchor A : (batch_size, n_user_features)
                        - Positive P : (batch_size, n_item_features)
                        - Negative N : (batch_size, n_item_features)
    """
    # constant values
    n_user_features = X_usr.shape[1]
    n_item_features = X_item.shape[1]

    # define user_list
    user_list = list(df.index.values)

    # initialise result
    triplets = [
        np.zeros((batch_size, n_user_features)),  # anchor
        np.zeros((batch_size, n_item_features)),  # positive
        np.zeros((batch_size, n_item_features)),  # negative
    ]
    user_ids = []
    p_ids = []
    n_ids = []

    for i in range(batch_size):
        # pick one random user for anchor
        anchor_id = random.choice(user_list)
        user_ids.append(anchor_id)

        # all possible positive/negative samples for selected anchor
        p_item_ids = get_pos(df, anchor_id)
        n_item_ids = get_neg(df, anchor_id)

        # pick one of the positve ids
        try:
            positive_id = random.choice(p_item_ids)
        except IndexError:
            positive_id = 0

        p_ids.append(positive_id)

        # pick the most similar negative id
        try:
            n_min = np.argmin(
                [
                    (cosine_dist(X_item[movie_id_to_index_map.get(positive_id)], X_item[movie_id_to_index_map.get(k)]))
                    for k in n_item_ids
                ]
            )
            negative_id = n_item_ids[n_min]
        except:
            try:
                negative_id = random.choice(n_item_ids)
            except IndexError:
                negative_id = 0

        n_ids.append(negative_id)

        # define triplet
        triplets[0][i, :] = X_usr[anchor_id][:]

        if positive_id == 0:
            triplets[1][i, :] = np.zeros((n_item_features,))
        else:
            triplets[1][i, :] = X_item[movie_id_to_index_map.get(positive_id)][:]

        if negative_id == 0:
            triplets[2][i, :] = np.zeros((n_item_features,))
        else:
            triplets[2][i, :] = X_item[movie_id_to_index_map.get(negative_id)][:]

    if return_cache:
        cache = {"users": user_ids, "positive": p_ids, "negative": n_ids}
        return triplets, cache

    return triplets

In [213]:
X_usr.shape, X_item.shape, ratings_matrix_df.shape

((6040, 23), (2339, 28), (6040, 2339))

In [91]:
get_triplets_hard(
    batch_size=16, X_usr=X_usr, X_item=X_item, df=ratings_matrix_df, return_cache=True
)

In [59]:
# Hyper parameters
evaluate_every = 20  # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 100  # No. of training iterations
n_val = 100  # how many one-shot tasks to validate on

In [94]:
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()

for i in range(1, n_iter + 1):
    triplets = get_triplets_hard(1, X_usr, X_item, ratings_matrix_df)
    loss = network_train.train_on_batch(triplets, None)
    n_iteration += 1
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print(
            f"[{n_iteration}] Time for {i} iterations: {(time.time() - t_start) / 60.0:.1f} mins, Train Loss: {loss}"
        )

# Serialise weights to HDF5
network_train.save_weights("/kaggle/working/network_train.h5")
network_predict.save_weights("/kaggle/working/network_predict.h5")
print("Saved model to disk")

Starting training process!
-------------------------------------


InvalidArgumentError: Graph execution error:

Detected at node training_model_1/item_embedding_3/GatherV2 defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelbase.py"